## Primera Versión
###### ...a nivel usuario sin clase

In [ ]:
import sqlite3

def crea_bbdd(bbdd,nombre_tabla, columnas):
    mi_conexion = sqlite3.connect(bbdd)
    mi_cursor = mi_conexion.cursor()

    # Construir la parte de la consulta SQL para las columnas
    columnas_sql = ', '.join([f'{key} {value}' for key, value in columnas.items()])
    
    # Crear la tabla
    mi_cursor.execute(f'''
        CREATE TABLE {nombre_tabla} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            {columnas_sql}
        )
    ''')
    
    mi_conexion.commit()
    mi_conexion.close()



def crea_diccionario_columnas():
    diccionario_columnas = {}
    
    cantidad_elementos = int(input('Introduce la cantidad de elementos: '))
    
    for _ in range(cantidad_elementos):
        nombre_columna = input('Introduce el nombre de la columna: ')
        tipo_columna = input(f'Introduce el tipo de la columna {nombre_columna} (por ejemplo, VARCHAR, INTEGER): ')
        diccionario_columnas[nombre_columna] = tipo_columna
    
    return diccionario_columnas

def inserta_datos(bbdd, nombre_tabla, columnas):
    mi_conexion = sqlite3.connect(bbdd)
    mi_cursor = mi_conexion.cursor()

    valores = tuple(input(f'Introduce {key}: ') for key in columnas.keys())
    mi_cursor.execute(f'INSERT INTO {nombre_tabla} VALUES (NULL, {", ".join(["?"] * len(columnas))})', valores)

    mi_conexion.commit()
    mi_conexion.close()

    
def menu():
    print('''
        1- Crear Base de datos y tabla
        2- Insertar datos en la tabla
        3- Salir''')



def averiguar_campos(nombre_bbdd, nombre_tabla):
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect(nombre_bbdd)
    cursor = conn.cursor()
    dic={}
    # Obtener los nombres de los campos de una tabla
    tabla = nombre_tabla
    cursor.execute(f"PRAGMA table_info({tabla})")
    campos = cursor.fetchall()

    # Imprimir los nombres de los campos
    for campo in campos[1:]:
        dic[campo[1]]=campo[2]

    # Cerrar la conexión
    conn.close()
    return dic




if __name__ == "__main__":
    while True:
        menu()
        quest= int(input('Introduce una opción: '))
        if quest == 1:
            nombre_bbdd = input('Introduce el nombre de la bbdd: ')
            nombre_bbdd += '.db'
            nombre_tabla = input('Introduce nombre de la tabla: ')
            diccionario_columnas=crea_diccionario_columnas()
            crea_bbdd(nombre_bbdd, nombre_tabla, diccionario_columnas)
        elif quest == 2:#aquí el diccionario columna hay que ver como averiguarlo porque está ligado a la creacion de bbdd
            nombre_bbdd = input('Introduce el nombre de la bbdd: ')
            nombre_bbdd += '.db'
            nombre_tabla = input('Introduce nombre de la tabla: ')
            diccionario_columnas=averiguar_campos(nombre_bbdd,nombre_tabla)
            while True:
                inserta_datos(nombre_bbdd, nombre_tabla, diccionario_columnas)
                seguir = input('Quieres introducir más datos (si/no): ')
                if seguir.lower() != 'si':
                    break
        elif quest ==3:
            break
        else:
            print('Has introducido un valor erróneo')
    

# Version 2
#### Versión creando una clase Bbdd

In [10]:
import sqlite3
import pandas as pd

class Bbdd:
    def __init__(self, nombre_bbdd='',nombre_tabla='',columnas={},csv_path=''):
        self.nombre_bbdd = nombre_bbdd
        self.nombre_tabla = nombre_tabla
        self.columnas = columnas
        self.csv_path =csv_path
        print('Se ha creado el inicio de una bbdd')
    
    
    def crear_bbdd(self):
        try:
            if self.nombre_bbdd == '':
                self.nombre_bbdd = input('Introduce el nombre de la bbdd sin la extensión : ')
                self.nombre_bbdd += '.db'
                self.nombre_tabla = input('Introduce nombre de la tabla: ')
            else:
                print('Ya existe una base de datos con este nombre')

            self.columnas = self.crea_diccionario_columnas()
            self.mi_conexion = sqlite3.connect(self.nombre_bbdd)
            self.mi_cursor = self.mi_conexion.cursor()

            # Construir la parte de la consulta SQL para las columnas
            self.columnas_sql = ', '.join([f'{key} {value}' for key, value in self.columnas.items()])

            # Crear la tabla
            self.mi_cursor.execute(f'''
                CREATE TABLE {self.nombre_tabla} (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    {self.columnas_sql}
                )
            ''')

            self.mi_conexion.commit()

        except sqlite3.Error as e:
            print(f"Error al conectar o ejecutar la consulta: {e}")

        finally:
            # Cerrar la conexión, independientemente de si hubo un error o no
            if 'mi_conexion' in locals():
                self.mi_conexion.close()
    
        
    def menu(self):
        print ( '''
        1- Crear Base de Datos
        2- Insertar datos en bbddd
        3- Convertir csv a bbdd
        4- Salir
        ''')
        
    def start(self):
        while True:
            self.menu()
            quest= int(input('Introduce una opción: '))
            if quest == 1:
                self.crear_bbdd()
            elif quest == 2:
                while True:
                    self.inserta_datos()
                    self.seguir = input('Quieres introducir más datos (si/no): ')
                    if self.seguir.lower() != 'si':
                        break
            elif quest == 3:
                self.crear_tabla_desde_csv()
                
            elif quest == 4:
                break
            else:
                print('Has introducido un valor erróneo.')
            
    def crea_diccionario_columnas(self):
        while True:
            try:
                self.cantidad_elementos = int(input('Introduce la cantidad de elementos: '))
                break
            except ValueError:
                print('Error: Ingresa un número entero para la cantidad de elementos.')

        for _ in range(self.cantidad_elementos):
            while True:
                try:
                    self.nombre_columna = input('Introduce el nombre de la columna: ')
                    self.tipo_columna = input(f'Introduce el tipo de la columna {self.nombre_columna} (por ejemplo, VARCHAR, INTEGER): ')
                    self.columnas[self.nombre_columna] = self.tipo_columna
                    break
                except ValueError:
                    print('Error: Ingresa un tipo de dato válido para la columna.')

        return self.columnas



    def averiguar_campos(self):
        try:
            # Conectar a la base de datos SQLite
            self.conn = sqlite3.connect(self.nombre_bbdd)
            self.cursor = self.conn.cursor()
            self.dic = {}

            # Obtener los nombres de los campos de una tabla
            self.tabla = self.nombre_tabla
            self.cursor.execute(f"PRAGMA table_info({self.tabla})")
            self.campos = self.cursor.fetchall()

            # Imprimir los nombres de los campos
            for campo in self.campos[1:]:
                self.dic[campo[1]] = campo[2]

            # Cerrar la conexión
            self.conn.close()

            return self.dic

        except sqlite3.Error as e:
            print(f"Error al conectar a la base de datos: {e}")
            self.menu()

        except sqlite3.OperationalError as e:
            print(f"Error al ejecutar la consulta: {e}")
            self.menu()

    
    
       
    def inserta_datos(self):
        if self.nombre_bbdd == '' and self.nombre_tabla == '':
            self.nombre_bbdd = input('Introduce el nombre de la bbdd sin la extensión: ')
            self.nombre_bbdd += '.db'
            self.nombre_tabla = input('Introduce nombre de la tabla: ')
        self.columnas=self.averiguar_campos()
        self.mi_conexion = sqlite3.connect(self.nombre_bbdd)
        self.mi_cursor = self.mi_conexion.cursor()

        self.valores = tuple(input(f'Introduce {key}: ') for key in self.columnas.keys())
        self.mi_cursor.execute(f'INSERT INTO {self.nombre_tabla} VALUES (NULL, {", ".join(["?"] * len(self.columnas))})', self.valores)

        self.mi_conexion.commit()
        self.mi_conexion.close()
        
    def crear_tabla_desde_csv(self):
        if self.csv_path == '':
            self.csv_path= input('Introduce el nombre del archivo csv sin la extensión: ')
        self.csv_path += '.csv'
        try:
            df = pd.read_csv(self.csv_path)
        except FileNotFoundError:
            print(f'El archivo {self.csv_path} no existe o no se encuentra en el directorio actual')
            
        if self.nombre_bbdd == '':
                self.nombre_bbdd = input('Introduce el nombre de la bbdd: ')
                self.nombre_bbdd += '.db'
                self.nombre_tabla = input('Introduce nombre de la tabla: ')
        else:
            print('Ya existe una base de datos con este nombre')
        self.conn = sqlite3.connect(self.nombre_bbdd)
        self.cursor = self.conn.cursor()

        self.crear_tabla_query = f"CREATE TABLE {self.nombre_tabla} ({', '.join([f'{column} TEXT' for column in df.columns])})"
        self.cursor.execute(self.crear_tabla_query)

        df.to_sql(self.nombre_tabla, self.conn, index=False, if_exists='replace')

        self.conn.commit()
        self.conn.close()

        
if __name__ == "__main__":
    bbdd_001=Bbdd()
    bbdd_002=Bbdd()
    bbdd_002.start()
    
    
        
        

Se ha creado el inicio de una bbdd
Se ha creado el inicio de una bbdd

        1- Crear Base de Datos
        2- Insertar datos en bbddd
        3- Convertir csv a bbdd
        4- Salir
        
Introduce una opción: 3
Introduce el nombre del archivo csv sin la extensión: leo
El archivo leo.csv no existe o no se encuentra en el directorio actual
El archivo leo.csv.csv no existe o no se encuentra en el directorio actual
El archivo leo.csv.csv.csv no existe o no se encuentra en el directorio actual
El archivo leo.csv.csv.csv.csv no existe o no se encuentra en el directorio actual
El archivo leo.csv.csv.csv.csv.csv no existe o no se encuentra en el directorio actual
El archivo leo.csv.csv.csv.csv.csv.csv no existe o no se encuentra en el directorio actual
El archivo leo.csv.csv.csv.csv.csv.csv.csv no existe o no se encuentra en el directorio actual
El archivo leo.csv.csv.csv.csv.csv.csv.csv.csv no existe o no se encuentra en el directorio actual
El archivo leo.csv.csv.csv.csv.csv.csv.csv.

OSError: [Errno 22] Invalid argument: 'leo.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv.csv'